<a href="https://colab.research.google.com/github/tafartech/fake-essay-detection/blob/main/Fake_Essay_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'llm-detect-ai-generated-text:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F61542%2F7516023%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240411%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240411T123959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D770c04806b8ff7aa08dcd4e6f018dfd8cb12a171019a3810b2a72425e5d2777ccb88c3e2f25536cd0c28b8d4ed2358251c802d20c35779d611363bb7054d50785af9c290be51924b4568753242d080b6e1eab305e46d13eb49820d40bd47d9aa75998478f7b8bcb440ce7aab32347b440ce13cf406722615b78a359ca3d974cb14f8a4a2c1d4da3aefb21bd53fa8cf0fbfc92fba23617845dd1d81d7a08812dd115eb8b32333d03593a7a06e0f5847ce32d1b624693e6075dd438e61963e54765afa89d0690397575bc90edd394fd1312db3343e4cc19c6cc9aee92084eb33f6fc6dba46d0334bf726836a1955efef873373d767496dcdf233415f507e89443d,daigt-misc:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3945154%2F6865136%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240411%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240411T123959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D261921bc4e57e592ea45fec19f25772f36163d540039423a72abd29fe8a433fdebedc5d58775c196f11f380aa9bdc71f6235ae1f656500a1bfa2cd27c09cac6c3cf91ee10e0d6721b22bf78db4ea909626306d2b868c0d7e1f0344b10f82a6844488c18bf15cb04cafa02cdc06bdae1751529e5cb6534a586faeb60ffbc9f25da64a2613c7bd42b31fb6f1b30cda63e3e039a5d8f6d858071cd797044b0d06807a701ca7a8be0440fe2cabd9c51b1783d9208ba3d746a91668cb440f1dc47d0b99dc459dab1360ebef199029412c86fea6d050e974736229e6d21f5a806ecf81e1855dacbda59c8adc9757b9fc2b9a4d6d85700d0b2ddf258a4b30ec471ee6e8,daigt-proper-train-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3942644%2F6890527%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240411%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240411T123959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8ac226be90a13e04ab2150bfa07304ec11c83499d28bb9de452b6b01ed0036999f11bb1f1635638d84673caa4041c333a7bb88be1c7d0b76214e6e741f95c7dd1dd85731660321cfbd30a4a658013663b47bc5d66d87735c9359d50926ea95f1cffd42f1cd791bd49e3ef09fa0e592c42a28510768aa7af647aaadf26277f80f69de06e8cf2ed32220e9bc90db80ab38b0c373218aeb79b9a361234649888d1dfb0e261b2397d8e06dd777b1cda59490637a48a5d3da1032b22399823c99ef1308f43d4f96a972ba7892dbadd2cf4cabb53b9abb9c870eab34da51da222dc0f6d352950012332fc574d285adc2b7ca87711820a58e1ba401eb14ba0f4ca96cde,daigt-external-train-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3960967%2F6901341%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240411%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240411T123959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dbca73bf71e1b27468e863a1762f1fe0424763858f58e0181169ebdfbb833fec2d8f8c9d3bca50f3f4c14d1730739a86a09e1fd9e3f67e994f5d97a89a073925b5a1d683582a25bb67b082bb6d42d0a42ef3c3d209574f44fa422a71932ad507ddd8f131d233cf5014f1116880ee38ad29a85a0e6bad9ee8a8c3b5173845a4b0fb821c58e2aff93535baa9cf0f9a9cbc251e4469b9001633d88ef75f5948cf93e5134259a25fa7e1b7880ed6ed0a84075274b015550e4182510caa323c364423723c464577f62cda4b9ef6ff0756e0250c6bb9ce3dd7dba989aa41f82ad02edae9b7061f8d68b157b231430e5cbc694e84be98c711e99bc4bf3c72a2e26123d7d,daigt-v2-train-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4005256%2F6977472%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240411%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240411T123959Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D7038270ffdcc44167bd3c8a4f2c500725b23a6e5fddf1634db97e00d2977ed01f757f7f8f770a7de8439c1840639fdcd1f7a5f257a4b28d0cd1dc8e48e73e26fd03174c91a7594c47ff01c36f10840daf229e6f6eaa6e3493a7cf48d2491685135a4e953f6f9e0002b250628909e710e254013586ca9419c6d97b6929d9993e763c507c40b01739387809688dd3f878f8ec44d14048a0e730f27c4f39c5e245f930bba13d40ac0151f7368d12ff0626c08e32679f0dce49bcfb21506e9a5a2d105efdd017a8fccd0458d43e24e0438bf370266b1709ab5bbfafbec09a43fbdfca4fdd1e598360690e11f6d14249156414a538cae14cf75519c5b660d3e9662fd'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')

# Import Libraries

In [ ]:
import sys
import gc
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from datasets import Dataset
from transformers import PreTrainedTokenizerFast

# Replace the PyArrow import with pickle
import pickle


# Load Data

In [ ]:
import pandas as pd

# Define file paths
test_path = '/kaggle/input/llm-detect-ai-generated-text/test_essays.csv'
sub_path = '/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv'
org_train_path = '/kaggle/input/llm-detect-ai-generated-text/train_essays.csv'
train_path = '/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv'

# Read CSV files
test = pd.read_csv(test_path)
sub = pd.read_csv(sub_path)
org_train = pd.read_csv(org_train_path)
train = pd.read_csv(train_path, sep=',')

# Optionally, you can print some information about the loaded data
print("Test Data Shape:", test.shape)
print("Submission Data Shape:", sub.shape)
print("Original Train Data Shape:", org_train.shape)
print("Train Data Shape:", train.shape)


In [ ]:
# Drop duplicate rows based on the 'text' column
train = train.drop_duplicates(subset=['text']).reset_index(drop=True)


# Configuration Parameter

In [ ]:
# Configuration parameters
LOWERCASE = False
VOCAB_SIZE = 30522

# Your code continues...

# For example, you can use these parameters like this:
if LOWERCASE:
    # Do something when LOWERCASE is True
    pass
else:
    # Do something when LOWERCASE is False
    pass



# Byte-Pair Encoding (BPE) tokenizer

In [ ]:
# Create a Dataset from the 'test' DataFrame
dataset = Dataset.from_pandas(test[['text']])

# Define train_corp_iter function
def train_corp_iter():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

# Creating Byte-Pair Encoding tokenizer
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)

# Use the train_corp_iter function to train the tokenizer
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)

# Create a PreTrainedTokenizerFast object
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)

# Tokenize texts for test and train datasets
tokenized_texts_test = [tokenizer.tokenize(text) for text in tqdm(test['text'].tolist())]
tokenized_texts_train = [tokenizer.tokenize(text) for text in tqdm(train['text'].tolist())]


# TF-IDF vectorization

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def dummy(text):
    return text

# Create TF-IDF vectorizer with vocabulary from test data
vectorizer = TfidfVectorizer(
    ngram_range=(3, 5),
    lowercase=False,
    sublinear_tf=True,
    analyzer='word',
    tokenizer=dummy,
    preprocessor=dummy,
    token_pattern=None,
    strip_accents='unicode'
)

# Fit on test data to get vocabulary
vectorizer.fit(tokenized_texts_test)
vocab = vectorizer.vocabulary_

# Print vocabulary (optional)
print(vocab)

# Use the obtained vocabulary for vectorizing train and test datasets
vectorizer = TfidfVectorizer(
    ngram_range=(3, 5),
    lowercase=False,
    sublinear_tf=True,
    vocabulary=vocab,
    analyzer='word',
    tokenizer=dummy,
    preprocessor=dummy,
    token_pattern=None,
    strip_accents='unicode'
)

# Transform train and test datasets
tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)

# Cleanup
del vectorizer
gc.collect()


In [ ]:
y_train = train['label'].values

# Create Submission

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier
import gc

# Check if the number of samples in the test set is less than or equal to 5
if len(test.text.values) <= 5:
    sub.to_csv('submission.csv', index=False)
else:
    # Define individual classifiers
    clf = MultinomialNB(alpha=0.02)
    sgd_model = SGDClassifier(max_iter=8500, tol=1e-4, loss="modified_huber")

    lgb_params = {
        'n_iter': 2480,
        'verbose': -1,
        'objective': 'cross_entropy',
        'metric': 'auc',
        'learning_rate': 0.0058,
        'colsample_bytree': 0.78,
        'colsample_bynode': 0.8,
        'lambda_l1': 4.56,
        'lambda_l2': 2.97,
        'min_data_in_leaf': 114,
        'max_depth': 24,
        'max_bin': 898
    }
    lgb = LGBMClassifier(**lgb_params)

    cat_params = {
        'iterations': 2010,
        'verbose': 0,
        'l2_leaf_reg': 6.659,
        'learning_rate': 0.00560,
        'subsample': 0.4,
        'allow_const_label': True,
        'loss_function': 'CrossEntropy'
    }
    cat = CatBoostClassifier(**cat_params)

    # Define weights for the voting classifier
    weights = [0.072, 0.31, 0.309, 0.309]

    # Create a voting classifier
    ensemble = VotingClassifier(estimators=[
        ('mnb', clf),
        ('sgd', sgd_model),
        ('lgb', lgb),
        ('cat', cat)
    ], weights=weights, voting='soft', n_jobs=-1)

    # Train the ensemble on the training set
    ensemble.fit(tf_train, y_train)

    # Make predictions on the test set
    final_preds = ensemble.predict_proba(tf_test)[:, 1]

    # Add the predictions to the submission DataFrame
    sub['generated'] = final_preds

    # Save the submission to a CSV file
    sub.to_csv('submission.csv', index=False)

    # Cleanup
    del ensemble, clf, sgd_model, lgb, cat
    gc.collect()


In [ ]:
# Display the submission results
print(sub.head())
